# **NETTOYAGE DE LA BASE DE DONNÉE DES VENTES**

In [1]:
import pandas as pd
import random
import string
from pathlib import Path
import re
import unicodedata

In [2]:
# === CONFIGURATION DES CHEMINS ===
BASE_DIR = Path('..')
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
FINAL_DIR = PROCESSED_DIR / 'final'
FINAL_DIR.mkdir(parents=True, exist_ok=True)

# === DÉFINITION DES FICHIERS D'ENTRÉE ===
fichiers_ventes = [
    {'fichier': 'ventes_2022.csv', 'annee': 2022},
    {'fichier': 'ventes_2023.csv', 'annee': 2023},
    {'fichier': 'ventes_2024.csv', 'annee': 2024}
]

In [4]:
# === FONCTIONS UTILES ===
def formater_numero(tel):
    if pd.isna(tel) or tel == "":
        return ""
    tel = str(tel).replace(" ", "").replace("+", "").replace("-", "").replace("(", "").replace(")", "")
    tel = re.sub(r'[^\d]', '', tel)
    if not tel:
        return ""
    if tel.startswith("237") and len(tel) >= 12:
        return "+237" + tel[3:12]
    elif tel.startswith("237") and len(tel) >= 9:
        reste = tel[3:]
        if not reste.startswith("6") and len(reste) in [8, 9]:
            reste = "6" + reste
        return "+237" + reste[:9]
    elif tel.startswith("6") and len(tel) >= 8:
        return "+237" + tel[:9]
    elif len(tel) >= 8 and not tel.startswith("6"):
        return "+237" + "6" + tel[:8]
    return ""

def generer_client_id():
    return "CLT" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

def generer_vente_id():
    return "VTE" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

def normaliser_nom(nom):
    if pd.isna(nom) or nom == "":
        return ""
    nom_normalise = unicodedata.normalize('NFD', str(nom).lower())
    nom_normalise = ''.join(c for c in nom_normalise if unicodedata.category(c) != 'Mn')
    nom_normalise = re.sub(r'\s+', ' ', nom_normalise.strip())
    return nom_normalise

In [5]:
# === CONSOLIDATION DES VENTES ===
print("=== CONSOLIDATION DES VENTES ===")
dataframes_consolides = []
for config in fichiers_ventes:
    fichier_path = PROCESSED_DIR / config['fichier']
    annee = config['annee']
    if fichier_path.exists():
        df = pd.read_csv(fichier_path)
        df.insert(0, 'annee', annee)
        dataframes_consolides.append(df)
        print(f"✅ {config['fichier']} chargé ({len(df)} lignes)")
    else:
        print(f"⚠️ Fichier introuvable : {fichier_path}")

if not dataframes_consolides:
    raise FileNotFoundError("Aucun fichier de ventes trouvé pour consolidation.")

df_ventes = pd.concat(dataframes_consolides, ignore_index=True)
print(f"✅ Total ventes consolidées : {len(df_ventes)} lignes")

=== CONSOLIDATION DES VENTES ===
✅ ventes_2022.csv chargé (102 lignes)
✅ ventes_2023.csv chargé (307 lignes)
✅ ventes_2024.csv chargé (240 lignes)
✅ Total ventes consolidées : 649 lignes


In [6]:
# === CHARGEMENT DES CLIENTS FINAUX ===
CLIENTS_FILE = FINAL_DIR / 'clients_final.csv'
if not CLIENTS_FILE.exists():
    raise FileNotFoundError(f"Fichier clients requis introuvable : {CLIENTS_FILE}")

df_clients = pd.read_csv(CLIENTS_FILE, dtype={"telephone": str, "telephone_secondaire": str})

In [7]:
# === STANDARDISATION DES NUMÉROS ===
df_ventes["telephone_original"] = df_ventes["telephone"].copy()
df_ventes["telephone"] = df_ventes["telephone"].apply(formater_numero)

# === MAPPING TÉLÉPHONE -> CLIENT ===
telephone_to_client = {}
for _, client in df_clients.iterrows():
    tel_principal = client.get("telephone", "")
    if pd.notna(tel_principal) and str(tel_principal).strip():
        telephone_to_client[str(tel_principal).strip()] = {
            "code_client": client.get("code_client", ""),
            "nom": client.get("nom", "")
        }

In [8]:
# === TRAITEMENT DES CLIENTS SANS NUMÉRO ===
ventes_sans_numero = df_ventes[(df_ventes["telephone"] == "") | (df_ventes["telephone"].isna())].copy()
nom_to_client_id = {}
clients_sans_numero_info = {}

if len(ventes_sans_numero) > 0:
    for _, vente in ventes_sans_numero.iterrows():
        nom_original = vente.get("nom", "")
        if pd.notna(nom_original) and str(nom_original).strip():
            nom_normalise = normaliser_nom(nom_original)
            if nom_normalise not in nom_to_client_id:
                nouveau_client_id = generer_client_id()
                nom_to_client_id[nom_normalise] = nouveau_client_id
                clients_sans_numero_info[nouveau_client_id] = {
                    "code_client": nouveau_client_id,
                    "nom": nom_original,
                    "telephone": "",
                    "telephone_secondaire": "",
                    "adresse": vente.get("adresse", ""),
                    "ville": vente.get("ville", ""),
                    "historique_commandes": "",
                    "nb_commandes": 0,
                    "alias": ""
                }

In [9]:
# === ASSOCIATION DES VENTES AVEC CLIENTS ===
def associer_client(row):
    telephone_vente = row.get("telephone", "")
    nom_vente = row.get("nom", "")
    if telephone_vente and telephone_vente in telephone_to_client:
        client_info = telephone_to_client[telephone_vente]
        return client_info["code_client"], client_info["nom"]
    elif pd.notna(nom_vente) and str(nom_vente).strip():
        nom_normalise = normaliser_nom(nom_vente)
        if nom_normalise in nom_to_client_id:
            return nom_to_client_id[nom_normalise], nom_vente
        else:
            new_id = generer_client_id()
            nom_to_client_id[nom_normalise] = new_id
            return new_id, nom_vente
    else:
        return generer_client_id(), ""

ids_noms = df_ventes.apply(associer_client, axis=1)
df_ventes["client_id"] = [i[0] for i in ids_noms]
df_ventes["nom"] = [i[1] for i in ids_noms]
df_ventes["vente_id"] = [generer_vente_id() for _ in range(len(df_ventes))]

In [10]:
# === EXPORT DES NOUVEAUX CLIENTS SANS NUMÉRO ===
if clients_sans_numero_info:
    for client_id in clients_sans_numero_info:
        clients_sans_numero_info[client_id]["nb_commandes"] = len(df_ventes[df_ventes["client_id"] == client_id])
    df_nouveaux_clients_sans_numero = pd.DataFrame(clients_sans_numero_info.values())
    output_clients_sans_numero = FINAL_DIR / "nouveaux_clients_sans_numero.csv"
    df_nouveaux_clients_sans_numero.to_csv(output_clients_sans_numero, index=False, encoding="utf-8")
    print(f"💾 Fichier exporté : {output_clients_sans_numero} ({len(df_nouveaux_clients_sans_numero)} clients)")

💾 Fichier exporté : ../data/processed/final/nouveaux_clients_sans_numero.csv (84 clients)


In [11]:
# === EXPORT DES VENTES ===
colonnes_finales = ["vente_id","client_id","nom","telephone","commande","montant","adresse","frais_livraison","bon_fidelite","ville","annee"]
for col in colonnes_finales:
    if col not in df_ventes.columns:
        df_ventes[col] = ""
df_ventes_final = df_ventes[colonnes_finales].copy()
VENTES_FILE = FINAL_DIR / "ventes_final.csv"
df_ventes_final.to_csv(VENTES_FILE, index=False, encoding="utf-8")
print(f"💾 Fichier ventes sauvegardé : {VENTES_FILE}")

💾 Fichier ventes sauvegardé : ../data/processed/final/ventes_final.csv


In [12]:
# === EXPORT FACTURES ===
df_ventes["montant"] = pd.to_numeric(df_ventes["montant"], errors="coerce").fillna(0)
factures = df_ventes[["vente_id","client_id","montant","ville","annee"]].copy()
factures.rename(columns={"vente_id":"facture_id","montant":"montant_total"}, inplace=True)
factures["date_facture"] = pd.to_datetime(factures["annee"].astype(str) + "-01-01")
factures["mode_paiement"] = "Inconnu"
factures = factures[["facture_id","date_facture","client_id","montant_total","ville","mode_paiement"]]
FACTURES_FILE = FINAL_DIR / "factures_final.csv"
factures.to_csv(FACTURES_FILE, index=False, encoding="utf-8")
print(f"💾 Fichier factures sauvegardé : {FACTURES_FILE}")

💾 Fichier factures sauvegardé : ../data/processed/final/factures_final.csv
